## Optimización bayesiana 
Para un kernel linear con k folds k=10**

In [18]:
pip install pandas scikit-learn optuna

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import PolynomialFeatures
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="optuna")
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [20]:
data = pd.read_csv(r"C:\Users\dayan\Downloads\brain_tumor_dataset.csv")
data.head()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Patient_ID           20000 non-null  int64  
 1   Age                  20000 non-null  int64  
 2   Gender               20000 non-null  object 
 3   Tumor_Type           20000 non-null  object 
 4   Tumor_Size           20000 non-null  float64
 5   Location             20000 non-null  object 
 6   Histology            20000 non-null  object 
 7   Stage                20000 non-null  object 
 8   Symptom_1            20000 non-null  object 
 9   Symptom_2            20000 non-null  object 
 10  Symptom_3            20000 non-null  object 
 11  Radiation_Treatment  20000 non-null  object 
 12  Surgery_Performed    20000 non-null  object 
 13  Chemotherapy         20000 non-null  object 
 14  Survival_Rate        20000 non-null  float64
 15  Tumor_Growth_Rate    20000 non-null 

In [26]:
# Convertimos el target a números 
le = LabelEncoder()
data['Tumor_Type_Encoded'] = le.fit_transform(data['Tumor_Type'])

print("Mapeo del Target 'Tumor_Type':")
for i, class_name in enumerate(le.classes_):
    print(f"  {class_name} -> {i}")

# Excluimos Patient_ID y las columnas de target
X = data.drop(['Patient_ID', 'Tumor_Type', 'Tumor_Type_Encoded'], axis=1)
y = data['Tumor_Type_Encoded'] # Nuestro target numérico

#División Train / Test 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y)

Mapeo del Target 'Tumor_Type':
  Benign -> 0
  Malignant -> 1


## Ingeniería de características 
Variables Numéricas: Les aplicamos StandardScaler (análisis de magnitud) y PolynomialFeatures (interacciones y transformaciones polinomiales).

Variables Categóricas (Nominales): Usamos OneHotEncoder.

Variables Categóricas (Ordinales): Usamos OrdinalEncoder para Stage, preservando el orden.

In [28]:
# Columnas numéricas (continuas)
numeric_features = ['Age', 'Tumor_Size', 'Survival_Rate', 'Tumor_Growth_Rate']

# Columnas categóricas (nominales, sin orden)
# Excluimos 'Stage' que es ordinal
categorical_features = [
    'Gender', 'Location', 'Histology', 'Symptom_1', 'Symptom_2', 'Symptom_3',
    'Radiation_Treatment', 'Surgery_Performed', 'Chemotherapy', 
    'Family_History', 'MRI_Result', 'Follow_Up_Required'
]

# Columnas categóricas (ordinales, con orden)
ordinal_features = ['Stage']
# Definimos el orden correcto
stage_order = [['I', 'II', 'III', 'IV']]

# Pipeline para Features Numéricas:
#  Escala (StandardScaler)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)) 
])
print("\nTransformer numérico creado (Scaler + PolyFeatures degree=2)")

# Transformer para Features Categóricas Nominales:
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
print("Transformer categórico creado (OneHotEncoder)")

# Transformer para Features Categóricas Ordinales:
ordinal_transformer = OrdinalEncoder(categories=stage_order, handle_unknown='error')
print("Transformer ordinal creado (OrdinalEncoder para 'Stage')")


# Unir todo en un ColumnTransformer 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ],
    remainder='passthrough' # Deja pasar columnas no especificadas (ninguna en este caso)
)


Transformer numérico creado (Scaler + PolyFeatures degree=2)
Transformer categórico creado (OneHotEncoder)
Transformer ordinal creado (OrdinalEncoder para 'Stage')


## Optimización Bayesiana (Optuna) para SVC
Aquí definimos la función objective que Optuna usará. En cada "trial", probará un valor diferente para el hiperparámetro C del SVC(kernel='linear').

El score que optimizamos es el Accuracy promedio de una validación cruzada de 10-folds (cv=10).

Una escala logarítmica prueba 0.01, 0.1, 1, 10, 100 de forma equilibrada. Una escala lineal probaría 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, ignorando casi por completo los valores bajos que son cruciales.

**¿Qué hace optuna?**

Define un Objetivo: Tú le das una función (objective) que le dice cómo medir si un modelo es "bueno" (en este caso, entrenar un SVC con 10-folds y devolver el accuracy promedio).

Define el Espacio de Búsqueda: Le dices qué hiperparámetros probar y en qué rangos (ej. C entre 0.01 y 100).

Prueba y Aprende: Optuna ejecuta tu función objective muchas veces (trials).

En cada "trial", prueba una combinación de hiperparámetros (ej. C=5.7).

Mide el resultado (accuracy).

Utiliza los resultados de todos los trials pasados para "aprender" qué rangos de C están dando mejores resultados.

Decide de forma inteligente cuál será el próximo valor de C que probará, enfocándose en las zonas más prometedoras.

Su objetivo final es encontrar la combinación de hiperparámetros que te da el mejor score posible (maximizar tu accuracy) en el menor número de intentos.

In [30]:
print("\n--- Parte 3: Iniciando Optimización Bayesiana para SVC ---")

#  Definir la función 'objective' para Optuna 
# Optuna llamará a esta función en cada 'trial'
def objective(trial):
    
    # Sugerir Hiperparámetros
    # Buscamos el mejor valor para 'C' (regularización)
    # Un rango logarítmico de 0.01 a 100 es un buen punto de partida, ¿Por qué logarítmica? eso lo respondo arriba
    param_c = trial.suggest_float('C', 1e-2, 100.0, log=True)
    
    # Crear el modelo SVC 
    # Usamos kernel='linear' como se pidió
    model = SVC(kernel='linear', C=param_c, random_state=42)
    
    # Crear el Pipeline completo
    # Encadenamos el preprocesamiento y el modelo
    pipeline_svc = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Evaluar el Pipeline con Cross-Validation (k=10)
    # Usamos 'accuracy' como métrica, como se pidió
    # n_jobs=-1 usa todos los cores para acelerar
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    score = cross_val_score(
        pipeline_svc, 
        X_train, 
        y_train, 
        cv=kfold, 
        scoring='accuracy', 
        n_jobs=-1
    )
    
    # Retornar el score promedio
    # Optuna intentará maximizar este valor
    return score.mean()

# Crear y ejecutar Optuna 
print("Creando estudio de Optuna (direction='maximize')...")
# Buscamos maximizar el 'accuracy'
study = optuna.create_study(direction='maximize')

# Correr la optimización
# n_trials = cuántas combinaciones probar
# 5 trials = 50 entrenamientos (5 trials * 10 K-Folds)
# Solo 5 trials porque toma alrededor de 5 minutos cada uno y se me termina suspendiendo la compu ya van como 5  veces que no termina el código 
n_trials_optuna = 5
print("Iniciando optimización con " + str(n_trials_optuna) + " trials...")
print("Esto puede tardar varios minutos...")

study.optimize(objective, n_trials=n_trials_optuna) 

print("\nOptimización Bayesiana completada")
print("Mejor score (Accuracy promedio en K-Fold) encontrado: " + str(study.best_value))
print("Mejores Hiperparámetros encontrados: " + str(study.best_params))

[I 2025-10-30 13:02:30,710] A new study created in memory with name: no-name-f2745b38-ccc1-421c-9fd8-2fdb7a14967c



--- Parte 3: Iniciando Optimización Bayesiana para SVC ---
Creando estudio de Optuna (direction='maximize')...
Iniciando optimización con 5 trials...
Esto puede tardar varios minutos...


[I 2025-10-30 13:12:58,233] Trial 0 finished with value: 0.4930624999999999 and parameters: {'C': 7.453470407484805}. Best is trial 0 with value: 0.4930624999999999.
[I 2025-10-30 13:15:53,166] Trial 1 finished with value: 0.4928125 and parameters: {'C': 2.4593136460699427}. Best is trial 0 with value: 0.4930624999999999.
[I 2025-10-30 13:16:51,384] Trial 2 finished with value: 0.49312500000000004 and parameters: {'C': 0.4637321929265878}. Best is trial 2 with value: 0.49312500000000004.
[I 2025-10-30 13:17:39,723] Trial 3 finished with value: 0.4932499999999999 and parameters: {'C': 0.2646431759990649}. Best is trial 3 with value: 0.4932499999999999.
[I 2025-10-30 13:18:11,695] Trial 4 finished with value: 0.49293750000000003 and parameters: {'C': 0.17000413575627277}. Best is trial 3 with value: 0.4932499999999999.



Optimización Bayesiana completada
Mejor score (Accuracy promedio en K-Fold) encontrado: 0.4932499999999999
Mejores Hiperparámetros encontrados: {'C': 0.2646431759990649}


Parte 4: Entrenamiento Final y Evaluación en Test
Finalmente, tomamos el mejor hiperparámetro C encontrado por Optuna, creamos un último pipeline, lo entrenamos con todos los datos de X_train y y_train, y medimos el Accuracy final en el set X_test que habíamos guardado.

In [32]:
print("\n--- Parte 4: Entrenando modelo final y evaluando en X_test ---")

# 1. Extraer los mejores parámetros
best_params = study.best_params
best_C = best_params['C']
print("Mejor 'C' encontrado: " + str(best_C))

final_model = SVC(kernel='linear', C=best_C, random_state=42)

# Crear el pipeline final
final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', final_model)
])

# Entrenar en todo X_train
print("Entrenando el modelo final en todo X_train...")
final_pipeline.fit(X_train, y_train)
print("Modelo final entrenado.")

# Predecir en el set de prueba (X_test)
y_pred_test = final_pipeline.predict(X_test)

# Calcular el Accuracy final en el set de prueba
final_acc = accuracy_score(y_test, y_pred_test)
print("  Accuracy final en el set de prueba (X_test): " + str(final_acc))

#  Mostrar reporte de clasificación detallado
#
print("\nReporte de Clasificación en X_test:")
# Usamos le.classes_ para mostrar los nombres 'Benign' y 'Malignant'
report = classification_report(y_test, y_pred_test, target_names=le.classes_)
print(report)

print(" Proceso Completado ")


--- Parte 4: Entrenando modelo final y evaluando en X_test ---
Mejor 'C' encontrado: 0.2646431759990649
Entrenando el modelo final en todo X_train...
Modelo final entrenado.
  Accuracy final en el set de prueba (X_test): 0.50725

Reporte de Clasificación en X_test:
              precision    recall  f1-score   support

      Benign       0.50      0.61      0.55      1994
   Malignant       0.51      0.41      0.45      2006

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.50      4000
weighted avg       0.51      0.51      0.50      4000

 Proceso Completado 
